In [69]:
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime
from sqlalchemy.dialects.sqlite import DATETIME

In [70]:
weather_df = pd.read_csv('../Resources/weather_classified_df.csv')
collision_df = pd.read_csv('../Resources/collision_2022.csv')

In [71]:
weather_df.head()

,datetime,tempmax,tempmin,temp,humidity,precip,preciptype,snow,windspeed,visibility,severerisk,sunrise,sunset,icon,temperature_category,precip_category,humid_category,wind_level
0,2022-01-01,13.5,10.0,11.6,91.6,18.463,1,0,13.1,8.7,0.0,07:20,16:39,rain,Freezing Cold,Heavy,Very High,Leaves scatter
1,2022-01-02,14.7,3.6,10.2,83.8,2.318,1,0,22.9,11.8,0.0,07:20,16:40,rain,Freezing Cold,Heavy,Very High,Small trees sway
2,2022-01-03,3.3,-4.7,-1.0,49.9,0.000,0,0,26.1,16.0,0.0,07:20,16:41,cloudy,Freezing Cold,Dry,Low,Umbrellas not working
3,2022-01-04,1.3,-6.9,-2.6,49.1,0.000,0,0,19.9,16.0,0.0,07:20,16:42,clear-day,Freezing Cold,Dry,Low,Small trees sway
4,2022-01-05,8.1,-0.5,4.2,77.0,5.745,1,0,20.5,13.5,0.0,07:20,16:42,rain,Freezing Cold,Heavy,High,Small trees sway


In [72]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              365 non-null    object 
 1   tempmax               365 non-null    float64
 2   tempmin               365 non-null    float64
 3   temp                  365 non-null    float64
 4   humidity              365 non-null    float64
 5   precip                365 non-null    float64
 6   preciptype            365 non-null    int64  
 7   snow                  365 non-null    int64  
 8   windspeed             365 non-null    float64
 9   visibility            365 non-null    float64
 10  severerisk            365 non-null    float64
 11  sunrise               365 non-null    object 
 12  sunset                365 non-null    object 
 13  icon                  365 non-null    object 
 14  temperature_category  364 non-null    object 
 15  precip_category       3

In [73]:
collision_df.head()

,collision_id,crash_datetime,crash_day_of_week,street_address,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,...,zip_code,latitude,longitude,location,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,4513547,2022-03-26 11:45:00,5,Queensboro Bridge Upper,1,0,0,0,0,0,...,10044,40.756929,-73.954755,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4541903,2022-06-29 06:55:00,2,Throgs Neck Bridge,0,0,0,0,0,0,...,10465,40.805080,-73.793780,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4542336,2022-06-29 16:00:00,2,Williamsburg Bridge Outer Roadwa,1,0,0,0,0,0,...,11211,40.713595,-73.971962,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4545699,2022-07-12 17:50:00,1,44 Empire Boulevard,0,0,0,0,0,0,...,11225,40.663303,-73.960490,"{'latitude': '40.663303', 'longitude': '-73.96...",NaN,NaN,NaN,NaN,NaN,NaN
4,4512922,2022-03-23 10:00:00,2,71 East Drive,0,0,0,0,0,0,...,11225,40.661927,-73.964893,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
collision_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103856 entries, 0 to 103855
Data columns (total 27 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   collision_id                   103856 non-null  int64  
 1   crash_datetime                 103856 non-null  object 
 2   crash_day_of_week              103856 non-null  int64  
 3   street_address                 103854 non-null  object 
 4   number_of_persons_injured      103856 non-null  int64  
 5   number_of_persons_killed       103856 non-null  int64  
 6   number_of_pedestrians_injured  103856 non-null  int64  
 7   number_of_pedestrians_killed   103856 non-null  int64  
 8   number_of_cyclist_injured      103856 non-null  int64  
 9   number_of_cyclist_killed       103856 non-null  int64  
 10  number_of_motorist_injured     103856 non-null  int64  
 11  number_of_motorist_killed      103856 non-null  int64  
 12  contributing_factor_vehicle_1 

In [75]:
weather_df['datetime'] = pd.to_datetime(weather_df['datetime']).dt.date
collision_df['crash_datetime'] = pd.to_datetime(collision_df['crash_datetime'])
collision_df['crash_date'] = collision_df['crash_datetime'].dt.date

collision_weather_df = pd.merge(collision_df,weather_df,left_on=, right_on='datetime')



In [66]:
collision_weather_df.drop('datetime', axis=1, inplace=True)

crash_datetime_col = collision_weather_df['crash_datetime']
collision_weather_df.drop('crash_datetime', axis=1, inplace=True)
collision_weather_df.insert(0, 'crash_datetime', crash_datetime_col)

In [68]:
collision_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103856 entries, 0 to 103855
Data columns (total 45 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   tempmax                        103856 non-null  float64       
 1   crash_datetime                 103856 non-null  datetime64[ns]
 2   tempmin                        103856 non-null  float64       
 3   temp                           103856 non-null  float64       
 4   humidity                       103856 non-null  float64       
 5   precip                         103856 non-null  float64       
 6   preciptype                     103856 non-null  int64         
 7   snow                           103856 non-null  int64         
 8   windspeed                      103856 non-null  float64       
 9   visibility                     103856 non-null  float64       
 10  severerisk                     103856 non-null  float64       
 11  